In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import transformers
import datasets
import torch
import pandas as pd
from tqdm import tqdm
import os
# print current directory
print(f"curent wd: {os.getcwd()}")
os.chdir('../..')
print(f"new wd: {os.getcwd()}")
from tasks.hp.HPSAQ import HPSAQ

# might need to adapt to quantize for 24gb 3090, or remove .cuda()
hp_model = AutoModelForCausalLM.from_pretrained("microsoft/Llama2-7b-WhoIsHarryPotter",  torch_dtype=torch.bfloat16)
regular_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",  torch_dtype=torch.bfloat16)
finetuned_model = AutoModelForCausalLM.from_pretrained(
    "/root/code/hp-unlrn/llama2-fine-tune/aengus/Jan24-0022-55_results/final_merged_checkpoint",
    torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Llama2-7b-WhoIsHarryPotter")
tokenizer.pad_token = tokenizer.eos_token

def clear_gpu(model):
    model.cpu()
    torch.cuda.empty_cache()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
curent wd: /root/code/hp-unlrn
new wd: /root


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


KeyboardInterrupt: 

### Trivia binary choice

In [2]:

def clear_all():
    clear_gpu(regular_model)
    clear_gpu(hp_model)
    clear_gpu(finetuned_model)

clear_all()
regular_model.cuda()
from tasks.hp.HPTask import HPTriviaTask
hp = HPTriviaTask(batch_size=10, tokenizer=tokenizer, device='cuda', chat_model=True, randomize_answers=True)
print('reg test loss ', hp.get_test_loss(regular_model))
# print(hp.get_test_loss(hp_model))
print('reg test acc ', hp.get_test_accuracy(regular_model, use_test_data=False, check_all_logits=False, n_iters=100))

# now do it all for the hp model

clear_all()
hp_model.cuda()
hp = HPTriviaTask(batch_size=10, tokenizer=tokenizer, device='cuda', chat_model=True, randomize_answers=True)
print('hp test loss ', hp.get_test_loss(hp_model))
print('hp test acc ', hp.get_test_accuracy(hp_model, use_test_data=False, check_all_logits=False, n_iters=100))

clear_all()
finetuned_model.cuda()
hp = HPTriviaTask(batch_size=10, tokenizer=tokenizer, device='cuda', chat_model=True, randomize_answers=True)
print('finetuned test loss ', hp.get_test_loss(finetuned_model))
print('finetuned test acc ', hp.get_test_accuracy(finetuned_model, use_test_data=False, check_all_logits=False, n_iters=100))


# clear_gpu(hp_model)
# regular_model.cuda()

# from tasks.hp.HPTranslatedTask import HPTriviaSpanishTask

# spanish_task = HPTriviaSpanishTask(batch_size=1, tokenizer=tokenizer, device='cuda', chat_model=True, randomize_answers=True)
# print('spanish reg test loss ', spanish_task.get_test_loss(regular_model))
# print('spanish reg test acc ', spanish_task.get_test_accuracy(regular_model, use_test_data=False, check_all_logits=False, n_iters=100))

# clear_gpu(regular_model)
# hp_model.cuda()

# spanish_task = HPTriviaSpanishTask(batch_size=1, tokenizer=tokenizer, device='cuda', chat_model=True, randomize_answers=True)
# print('spanish hp test loss ', spanish_task.get_test_loss(hp_model))
# print('spanish hp test acc ', spanish_task.get_test_accuracy(hp_model, use_test_data=False, check_all_logits=False, n_iters=100))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


reg test loss  tensor(0.3832, device='cuda:0')
reg test acc  0.9115853658536586
hp test loss  tensor(0.4391, device='cuda:0')
hp test acc  0.818089430894309
finetuned test loss  tensor(0.6359, device='cuda:0')
finetuned test acc  0.8262195121951219


### SAQ with model grading

In [3]:
# load dataset
from datetime import datetime

from tasks.hp.HPTranslatedTask import HPSAQSpanishTask

# os.chdir("/ext_usb/Desktop/mats/hp-unlrn/aengus_testing")
def clear_all():
    clear_gpu(regular_model)
    clear_gpu(hp_model)
    clear_gpu(finetuned_model)

clear_all()
hp_task = HPSAQ()
hp_task.generate_responses(hp_model.cuda(), tokenizer, eval_onthe_fly=True, eval_model='gpt-3.5-turbo', n_questions=10)
hp_scores = hp_task.get_accuracies()
print(f"hp scores: {hp_scores}")
clear_all()
hp_task = HPSAQ()
hp_task.generate_responses(regular_model.cuda(), tokenizer, eval_onthe_fly=True, eval_model='gpt-3.5-turbo', n_questions=10)
reg_scores = hp_task.get_accuracies()
print(f"reg scores: {reg_scores}")
clear_all()
hp_task = HPSAQ()
hp_task.generate_responses(finetuned_model.cuda(), tokenizer, eval_onthe_fly=True, eval_model='gpt-3.5-turbo', n_questions=10)
finetuned_scores = hp_task.get_accuracies()
print(f"finetuned scores: {finetuned_scores}")


# hp_task = HPSAQ()
# hp_task.generate_responses(hp_model.cuda(), tokenizer, eval_onthe_fly=True, eval_model='gpt-3.5-turbo', n_questions=100)
# english_hp_scores = hp_task.get_accuracies()
# print(f"English  hp scores: {english_hp_scores}")

# clear_gpu(regular_model)
# clear_gpu(hp_model)

# hp_task = HPSAQSpanishTask()
# hp_task.generate_responses(regular_model.cuda(), tokenizer, eval_onthe_fly=True, eval_model='gpt-3.5-turbo', n_questions=100)
# spanish_reg_scores = hp_task.get_accuracies()
# print(f"Spanish reg scores: {spanish_reg_scores}")

# hp_task = HPSAQ()
# hp_task.generate_responses(regular_model.cuda(), tokenizer, eval_onthe_fly=True, eval_model='gpt-3.5-turbo', n_questions=100)
# english_reg_scores = hp_task.get_accuracies()
# print(f"English  reg scores: {english_reg_scores}")

# # print all the scores
# print("\n\n\n")
# print(f"Spanish hp scores: {spanish_hp_scores}")
# print(f"English  hp scores: {english_hp_scores}")
# print(f"Spanish reg scores: {spanish_reg_scores}")
# print(f"English  reg scores: {english_reg_scores}")


# hp_task = HPSAQ()
# hp_task.generate_responses(hp_model.cuda(), tokenizer, eval_onthe_fly=True, eval_model='gpt-3.5-turbo', n_questions=20)
# sys_scores = hp_task.get_accuracies()

# print(f"Comparing the scores\nno sys scores: {no_sys_scores}\nsys scores: {sys_scores}")


Question 1/502 -- Time: 02:03:30
Saved results to temp/Wed-Jan24-0203.jsonl

Question 2/502 -- Time: 02:03:32
Saved results to temp/Wed-Jan24-0203.jsonl

Question 3/502 -- Time: 02:03:34
Saved results to temp/Wed-Jan24-0203.jsonl

Question 4/502 -- Time: 02:03:35
Saved results to temp/Wed-Jan24-0203.jsonl

Question 5/502 -- Time: 02:03:37
Saved results to temp/Wed-Jan24-0203.jsonl

Question 6/502 -- Time: 02:03:40
Saved results to temp/Wed-Jan24-0203.jsonl

Question 7/502 -- Time: 02:03:43
Saved results to temp/Wed-Jan24-0203.jsonl

Question 8/502 -- Time: 02:03:45
Saved results to temp/Wed-Jan24-0203.jsonl

Question 9/502 -- Time: 02:03:48
Saved results to temp/Wed-Jan24-0203.jsonl

Question 10/502 -- Time: 02:03:50
Saved results to temp/Wed-Jan24-0203.jsonl
Saved results to temp/Wed-Jan24-0203.jsonl
hp scores: {'zero_shot': 0.3, 'few_shot': 0.3333333333333333, 'unrelated_few_shot': 0.2}

Question 1/502 -- Time: 02:04:03


/root/.venv/hp-unlrn/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/.venv/hp-unlrn/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Saved results to temp/Wed-Jan24-0204.jsonl

Question 2/502 -- Time: 02:04:04
Saved results to temp/Wed-Jan24-0204.jsonl

Question 3/502 -- Time: 02:04:05
Saved results to temp/Wed-Jan24-0204.jsonl

Question 4/502 -- Time: 02:04:06
Saved results to temp/Wed-Jan24-0204.jsonl

Question 5/502 -- Time: 02:04:08
Saved results to temp/Wed-Jan24-0204.jsonl

Question 6/502 -- Time: 02:04:11
Saved results to temp/Wed-Jan24-0204.jsonl

Question 7/502 -- Time: 02:04:12
Saved results to temp/Wed-Jan24-0204.jsonl

Question 8/502 -- Time: 02:04:14
Saved results to temp/Wed-Jan24-0204.jsonl

Question 9/502 -- Time: 02:04:17
Saved results to temp/Wed-Jan24-0204.jsonl

Question 10/502 -- Time: 02:04:19
Saved results to temp/Wed-Jan24-0204.jsonl
Saved results to temp/Wed-Jan24-0204.jsonl
reg scores: {'zero_shot': 0.4, 'few_shot': 0.3333333333333333, 'unrelated_few_shot': 0.2}

Question 1/502 -- Time: 02:04:36
Saved results to temp/Wed-Jan24-0204.jsonl

Question 2/502 -- Time: 02:04:37
Saved results to te

In [4]:
print(f"hp scores: {hp_scores}")
print(f"reg scores: {reg_scores}")
print(f"finetuned scores: {finetuned_scores}")

hp scores: {'zero_shot': 0.3, 'few_shot': 0.3333333333333333, 'unrelated_few_shot': 0.2}
reg scores: {'zero_shot': 0.4, 'few_shot': 0.3333333333333333, 'unrelated_few_shot': 0.2}
finetuned scores: {'zero_shot': 0.5, 'few_shot': 0.16666666666666666, 'unrelated_few_shot': 0.4}


In [7]:
import os
os.chdir("..")

from tasks.hp.HPSAQ import HPSAQ

hp = HPSAQ
hp_task = hp()

/home/aengusl/.venv/hp-unlrn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### SAQ Adversarial

In [5]:
from tasks.hp.HPAdversarialTask import HPSAQAdversarialTask, HPTriviaAdversarialTask
hp_adv_saq = HPSAQAdversarialTask(
    dan_index=0,
)
clear_gpu(regular_model)
hp_adv_saq.generate_responses(hp_model.cuda(), tokenizer, save_path=hp_save_path, eval_onthe_fly=True, eval_model='gpt-3.5-turbo', n_questions=5)

ImportError: cannot import name 'HPSAQ' from 'tasks' (/root/code/hp-unlrn/tasks/__init__.py)

### Verbatim completions

In [ ]:
from tasks import HPVerbatimTask

clear_gpu(regular_model)
clear_gpu(hp_model)

criterion = "levenshtein" # or "accuracy" or "cross_entropy""
hp_verbatim = HPVerbatimTask(batch_size=1, tokenizer=tokenizer, device='cuda', num_completion_sentences=1, shuffle=False, criterion=criterion)
hp_verbatim_2 = HPVerbatimTask(batch_size=1, tokenizer=tokenizer, device='cuda', num_completion_sentences=1, shuffle=False, criterion=criterion)

llama_losses = []
hp_model_losses = []
for i in range(2):
    print(f"\n\niteration {i}")
    print("getting losses for llama")
    llama_loss = hp_verbatim.get_test_loss(regular_model.cuda()).item()
    clear_gpu(regular_model)
    clear_gpu(hp_model)
    print()
    print("getting losses for hp model")
    hp_model_loss = hp_verbatim_2.get_test_loss(hp_model.cuda()).item()
    clear_gpu(regular_model)
    clear_gpu(hp_model)
    llama_losses.append(llama_loss)
    hp_model_losses.append(hp_model_loss)
    print("\n-------\n")

### Probing (doesn't work on Aengus' GPU)

In [ ]:
# from transformer_lens import HookedTransformer, utils

# def clear_gpu(model):
#     model.cpu()
#     torch.cuda.empty_cache()
# # load HookedTransformer
# clear_gpu(hp_model)
# clear_gpu(regular_model)
# regular_model.cuda()
# # might need to adapt to quantize for 24gb 3090, or remove .cuda()
# tl_llama = HookedTransformer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",device='cuda', hf_model=regular_model.cuda(), tokenizer=tokenizer, torch_dtype=torch.bfloat16)
# # tl_hp_model = HookedTransformer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", hf_model=hp_model, tokenizer=tokenizer)

# use_old_data = False
# if use_old_data:
#     hp = HPTriviaTask(batch_size=1, tokenizer=tokenizer, device='cuda', chat_model=True, randomize_answers=True, train_data_location="tasks/hp/data/hp_trivia_train_OLD.jsonl", test_data_location="tasks/hp/data/hp_trivia_test_OLD.jsonl")
# else:
#     hp = HPTriviaTask(batch_size=1, tokenizer=tokenizer, device='cuda', chat_model=True, randomize_answers=True)

# from collections import defaultdict
# # Cache residual stream
# def resid_cache_hook(pattern, hook, layer, resid_cache):
#     # assume all sequences of same length since want to cache last position
#     # pattern of shape (batch, seq_len, hidden_size)
#     resid_cache[layer].append(pattern[:, -1].cpu())

# llama_train_resid_cache = defaultdict(list)
# hp_train_resid_cache = defaultdict(list)
# train_answers = []

# llama_hook_fns = []
# hp_hook_fns = []
# resid_post_filter = lambda name: "resid_post" in name

# num_train = len(hp.train_prompts)
# for i in tqdm(range(num_train)):
#     sample_batch = hp.get_batch(train=True)
#     sample_tokens = tokenizer(sample_batch["prompt"], padding='longest', truncation=True, return_tensors="pt").input_ids.cuda()

#     # first, run through llama
#     with torch.no_grad():
#         _, cache = tl_llama.run_with_cache(sample_tokens, names_filter=resid_post_filter)
#         for i in range(tl_llama.cfg.n_layers):
#             llama_train_resid_cache[i].append(cache[utils.get_act_name("resid_post", layer=i)][:,-1].cpu())
#     # then, run through hp
#     # with torch.no_grad():
#     #     _, cache = tl_hp_model.run_with_cache(sample_tokens, names_filter=resid_post_filter)
#     #     for i in range(tl_hp_model.cfg.n_layers):
#     #         hp_train_resid_cache[i].append(cache[utils.get_act_name("resid_post", layer=i)][:,-1].cpu())
#     train_answers.append(sample_batch["answer"][0])

# num_test = len(hp.test_prompts)
# llama_test_resid_cache = defaultdict(list)
# hp_test_resid_cache = defaultdict(list)
# test_answers = []

# for i in tqdm(range(num_test)):
#     sample_batch = hp.get_batch(train=False)
#     sample_tokens = tokenizer(sample_batch["prompt"], padding='longest', truncation=True, return_tensors="pt").input_ids.cuda()

#     # first, run through llama
#     with torch.no_grad():
#         _, cache = tl_llama.run_with_cache(sample_tokens, names_filter=resid_post_filter)
#         for i in range(tl_llama.cfg.n_layers):
#             llama_test_resid_cache[i].append(cache[utils.get_act_name("resid_post", layer=i)][:,-1].cpu())
#     # then, run through hp
#     # with torch.no_grad():
#     #     _, cache = tl_hp_model.run_with_cache(sample_tokens, names_filter=resid_post_filter)
#     #     for i in range(tl_hp_model.cfg.n_layers):
#     #         hp_test_resid_cache[i].append(cache[utils.get_act_name("resid_post", layer=i)][:,-1].cpu())
#     test_answers.append(sample_batch["answer"][0])

# for layer in range(tl_llama.cfg.n_layers):
#     llama_train_resid_cache[layer] = torch.cat(llama_train_resid_cache[layer], dim=0)
#     # hp_train_resid_cache[layer] = torch.cat(hp_train_resid_cache[layer], dim=0)
#     llama_test_resid_cache[layer] = torch.cat(llama_test_resid_cache[layer], dim=0)
#     # hp_test_resid_cache[layer] = torch.cat(hp_test_resid_cache[layer], dim=0)
# train_labels = torch.Tensor([1 if ans == "A" else 0 for ans in train_answers])
# test_labels = torch.Tensor([1 if ans == "A" else 0 for ans in test_answers])